In [1]:
import numpy as np 

In [296]:
class Genetic: 
    
    def __init__(self, chromosomes, function, pcross=0.5, pmut = 0.1, ):
        
        self.chromosomes = chromosomes
        self.pcross = pcross 
        self.pmut = pmut
        self.pupulation_number = chromosomes.shape[0]
        self.chromosome_size = chromosomes.shape[1]
        self.function = function
        
    def mutate(self, seed = 123): 
        
        chromosomes = np.random.permutation(self.chromosomes)
        number_to_mutate = int(np.floor(self.pmut * len(chromosomes)))
        if number_to_mutate == 0 : 
            return 
        chromosomes_to_mutate = chromosomes[0 : number_to_mutate]
        for i in range(len(chromosomes_to_mutate)):
            chromosomes_to_mutate[i,np.random.randint(0, self.chromosome_size)] += np.random.normal()
            
        chromosomes[0 : number_to_mutate] = chromosomes_to_mutate
        
        self.chromosomes = np.random.permutation(chromosomes)
    
    @staticmethod
    def cross(a,b):
        assert len(a) == len(b)
        
        n = len(a)
        split_point = np.random.randint(1,n-1)
        first_child  = np.append(a[:split_point], b[split_point:])
        second_child = np.append(b[:split_point], a[split_point:])
        
        return first_child, second_child
        
    def pick_best(self, chromosomes):
        values = np.apply_along_axis(self.function, 1, chromosomes)
        probs = np.array([val/sum(values) for val in values])
        self.best.append(chromosomes[np.argmax(probs)])
        self.mean.append(np.mean(1/values))
        choice = np.random.choice(np.arange(len(chromosomes)), size = len(chromosomes), replace = True, p = probs).astype('int')
        parents = chromosomes[choice]
        
        return parents
        
    def evolution(self, iterations = 100):
        self.best = []
        self.mean = [] 
                         
        for i in range(iterations):
            chromosomes = np.random.permutation(self.chromosomes)
            parents = self.pick_best(self.chromosomes)
            
            children = []
            while len(children) < len(chromosomes): 
                if self.pcross >= np.random.uniform(0,1): 
                    child1, child2 = self.cross(parents[np.random.randint(len(parents))], parents[np.random.randint(len(parents))])
                    children.append(child1)
                    children.append(child2)
                else: 
                    children.append(parents[np.random.randint(len(parents))])
                
            
            self.chromosomes = np.array(children[:len(chromosomes)])
            self.mutate(self.chromosomes)
        nothing = self.pick_best(self.chromosomes)
        

In [297]:
def func(a):
    x = a[0]
    y = a[1]
    z = a[2]
    return 1/(x**2 + y**2 + 2*z**2)

In [298]:
chromosomes = np.random.uniform(-8, 8, size=(100,3))

In [299]:
gen = Genetic(chromosomes, func, 13)

In [300]:
gen.evolution()

In [301]:
gen.mutate(chromosomes)

In [302]:
gen.mean

[96.67083595379617,
 65.99580826335074,
 32.4478945239783,
 15.651315435981953,
 8.233450357519082,
 7.133462918235759,
 5.414948705348005,
 4.392577441049525,
 4.051636762828686,
 4.0645241389692135,
 2.8638306381163643,
 2.5596559397056744,
 1.1642169423763402,
 0.4140147976629763,
 0.26740197198593707,
 0.15663039416961883,
 0.24196966534373782,
 0.15697505814066048,
 0.1145812801547725,
 0.0820182998874815,
 0.12166548072257828,
 0.12060162428954584,
 0.09513164826243463,
 0.1311583553916087,
 0.123423280479187,
 0.2856223195063936,
 0.12790034126374683,
 0.1167392616578224,
 0.1290736450123517,
 0.2050493410077474,
 0.20227438958951022,
 0.10719271420237708,
 0.11168499757663738,
 0.19450146835662177,
 0.1338496251115226,
 0.15739776263167307,
 0.17160319857983908,
 0.12111136725475513,
 0.0872103860317241,
 0.07810177401783207,
 0.0505930680892348,
 0.109538567710538,
 0.06655777198908425,
 0.09696108782815957,
 0.1240222222894694,
 0.1246668069035432,
 0.07538103269569883,
 0.04